In [1]:
# @title CELL 0 — MOUNT GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# @title CUDA MEMORY CONF
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [3]:
# @title CELL 1 — INSTALL LIBS
!pip install -q transformers datasets peft accelerate bitsandbytes


In [4]:
# @title CELL 2 — IMPORTS & PATHS
import os, glob, json
import torch

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model

TRAIN_DIR = "/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data"
MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"  # hoặc 3B


In [5]:
# @title CHECK TRAIN FILES (COLAB)
import glob, os

files = glob.glob(f"{TRAIN_DIR}/*.jsonl")
print("Số file train:", len(files))
print(files[:5])

for f in files[:3]:
    print(f, "size =", os.path.getsize(f))


Số file train: 50
['/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_23_raw.jsonl', '/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_35_raw.jsonl', '/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_50_raw.jsonl', '/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_05_raw.jsonl', '/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_18_raw.jsonl']
/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_23_raw.jsonl size = 0
/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_35_raw.jsonl size = 0
/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_50_raw.jsonl size = 0


In [6]:
# @title CELL 3 — LOAD DATASET
files = [f for f in glob.glob(f"{TRAIN_DIR}/*.jsonl") if os.path.getsize(f) > 0]
print("Số file dùng train:", len(files))

dataset = load_dataset(
    "json",
    data_files=files,
    split="train"
)

print("Số mẫu:", len(dataset))
print(dataset[0])


Số file dùng train: 27


Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

Số mẫu: 3200
{'messages': [{'role': 'user', 'content': 'Tao lại tạch phỏng vấn nữa rồi mày ơi. Đây là công ty thứ 10 từ chối tao trong tháng này. Tao thấy mình vô dụng quá.'}, {'role': 'assistant', 'content': "Trời ơi, 10 công ty thôi mà, nhằm nhò gì! Mày đang sưu tập bộ 'Tem từ chối' để đổi quà hay sao? Vô dụng cái gì, mày là 'dũng sĩ diệt niềm tin của HR' thì có. Họ từ chối mày vì lý do gì, hay lại văn mẫu 'Em rất tốt nhưng anh rất tiếc'?"}, {'role': 'user', 'content': "Họ bảo 'chưa phù hợp văn hóa'. Tao cũng chả hiểu văn hóa họ là gì mà tao không hợp."}, {'role': 'assistant', 'content': "Văn hóa 'làm không lương' hay văn hóa 'sếp luôn đúng'? Cái lý do đó nó mông lung như một trò đùa vậy á. Chắc họ thấy mày 'sáng' quá sợ vào làm lu mờ hào quang của sếp thôi. Đừng buồn, công ty đó không có 'phước' rước mày về thôi. Giờ mày đang ngồi ủ rũ hay đang rải CV tiếp?"}, {'role': 'user', 'content': 'Đang nằm dài trên giường, chán chả muốn làm gì. Cảm giác như cả thế giới quay lưng.'}, {'role':

In [7]:
# @title CELL 3.5 — SANITIZE RAW DATASET (BẮT BUỘC)

BAD_KEYWORDS = [
    "Cutting Knowledge Date",
    "Today Date",
    "system\n",
    "assistant\n",
    "user\n"
]

def sanitize_example(example):
    clean_messages = []
    for m in example["messages"]:
        content = m["content"]
        # loại bỏ hoàn toàn mẫu có meta/system
        if any(bad in content for bad in BAD_KEYWORDS):
            return None
        clean_messages.append(m)
    return {"messages": clean_messages}

dataset = dataset.map(sanitize_example)
dataset = dataset.filter(lambda x: x is not None)

print("Số mẫu sau sanitize:", len(dataset))


Số mẫu sau sanitize: 3200


In [8]:
# @title CELL 4 — TOKENIZER (OVERRIDE CHAT TEMPLATE)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = 768

# 🔴 VÔ HIỆU HÓA CHAT TEMPLATE MẶC ĐỊNH
tokenizer.chat_template = None


In [9]:
# @title CELL 5 — FORMAT CHAT (FINAL, NO TEMPLATE)

SYSTEM_TEXT = "Bạn là một người bạn thân, nói chuyện tự nhiên, không dạy đời."

def formatting_prompts_func(example):
    parts = [f"<|system|>\n{SYSTEM_TEXT}\n"]

    for m in example["messages"]:
        if m["role"] == "user":
            parts.append(f"<|user|>\n{m['content']}\n")
        elif m["role"] == "assistant":
            parts.append(f"<|assistant|>\n{m['content']}\n")

    return {"text": "".join(parts)}

dataset = dataset.map(formatting_prompts_func)

print(dataset[0]["text"])


Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

<|system|>
Bạn là một người bạn thân, nói chuyện tự nhiên, không dạy đời.
<|user|>
Tao lại tạch phỏng vấn nữa rồi mày ơi. Đây là công ty thứ 10 từ chối tao trong tháng này. Tao thấy mình vô dụng quá.
<|assistant|>
Trời ơi, 10 công ty thôi mà, nhằm nhò gì! Mày đang sưu tập bộ 'Tem từ chối' để đổi quà hay sao? Vô dụng cái gì, mày là 'dũng sĩ diệt niềm tin của HR' thì có. Họ từ chối mày vì lý do gì, hay lại văn mẫu 'Em rất tốt nhưng anh rất tiếc'?
<|user|>
Họ bảo 'chưa phù hợp văn hóa'. Tao cũng chả hiểu văn hóa họ là gì mà tao không hợp.
<|assistant|>
Văn hóa 'làm không lương' hay văn hóa 'sếp luôn đúng'? Cái lý do đó nó mông lung như một trò đùa vậy á. Chắc họ thấy mày 'sáng' quá sợ vào làm lu mờ hào quang của sếp thôi. Đừng buồn, công ty đó không có 'phước' rước mày về thôi. Giờ mày đang ngồi ủ rũ hay đang rải CV tiếp?
<|user|>
Đang nằm dài trên giường, chán chả muốn làm gì. Cảm giác như cả thế giới quay lưng.
<|assistant|>
Thế giới quay lưng thì mày... cù lét thế giới cái cho nó quay 

In [10]:
# @title CELL 6 — TOKENIZE + LABELS

def tokenize_function(batch):
    tokens = tokenizer(
        batch["text"],
        truncation=True,
        max_length=768,        # 🔴 GIẢM TỪ 1024
        padding=False          # 🔴 KHÔNG PAD
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["messages", "text"]
)

print(dataset[0].keys())


Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [11]:
# @title CELL 7 — MODEL + LORA + MEMORY FIX

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# 🔒 Giảm VRAM mạnh
model.gradient_checkpointing_enable()
model.config.use_cache = False

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


`torch_dtype` is deprecated! Use `dtype` instead!


trainable params: 3,407,872 || all params: 1,239,222,272 || trainable%: 0.2750


In [12]:
# @title CELL 8 — TRAINER

from transformers import TrainingArguments, DataCollatorForSeq2Seq

training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=2,
    per_device_train_batch_size=2,   # 🔒 batch nhỏ
    gradient_accumulation_steps=8,   # 🔒 batch hiệu dụng = 16
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=500,
    bf16=True,
    report_to="none",
    optim="paged_adamw_8bit"
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,              # 🔒 padding động
    label_pad_token_id=-100    # 🔒 mask padding khỏi loss
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)


The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
# @title CELL 9 — TRAIN
trainer.train()


Step,Training Loss


In [ ]:
# @title CELL 10 — SAVE
trainer.model.save_pretrained("./lora_model")
tokenizer.save_pretrained("./lora_model")


In [ ]:
# @title CELL TEST FINAL — RELOAD & INFERENCE (FIX LỖI KÝ TỰ LẠ)
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Định nghĩa đường dẫn model vừa lưu (ở Cell 10)
ADAPTER_PATH = "./lora_model"

print("⏳ Đang dọn dẹp VRAM và load lại model sạch...")

# 2. Xóa model/trainer cũ để giải phóng VRAM (Quan trọng trên Colab Free)
try:
    del model, trainer
except:
    pass
torch.cuda.empty_cache()

# 3. Load lại Base Model (Llama-3.2-1B-Instruct)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# 4. Load LoRA Adapter và gộp vào base model
# (Chỉ gộp khi inference để đảm bảo tốc độ tốt nhất)
model_inference = PeftModel.from_pretrained(base_model, ADAPTER_PATH)

# 🔴 QUAN TRỌNG: Chuyển sang chế độ EVAL để tắt Dropout
model_inference.eval()
model_inference.config.use_cache = True

# 5. Load Tokenizer từ folder đã lưu
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH)

# 6. Định nghĩa Token kết thúc (Terminators) cho Llama 3
# Nếu thiếu cái này, model sẽ nói không ngừng hoặc sinh ký tự lạ
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# 7. Tạo Prompt Test
messages = [
    {
        "role": "system",
        # System prompt đúng với dữ liệu bạn đã train
        "content": "Bạn là một người bạn thân thiết, cực kỳ hài hước, lầy lội và hay 'cà khịa' nhưng thực tâm rất quan tâm đến bạn mình. Bạn luôn xưng hô 'tao - mày', dùng ngôn ngữ đời thường."
    },
    {
        "role": "user",
        "content": "Hôm nay tao mệt quá, đi làm về chỉ muốn nằm im."
    }
]

# Format prompt theo chuẩn Chat Template
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)

# 8. Sinh câu trả lời (Inference)
print("\n🤖 AI đang suy nghĩ (đã fix lỗi)...")
with torch.no_grad():
    outputs = model_inference.generate(
        **inputs,
        max_new_tokens=256,        # Độ dài câu trả lời tối đa
        do_sample=True,
        temperature=0.7,           # 0.7 giúp câu trả lời sáng tạo nhưng vẫn logic
        top_p=0.9,
        repetition_penalty=1.1,    # Tránh lặp từ
        eos_token_id=terminators,  # 🔴 Ngắt câu đúng chỗ
        pad_token_id=tokenizer.eos_token_id
    )

# Giải mã và in kết quả
response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)

print("=" * 50)
print("🗣️ KẾT QUẢ:", response)
print("=" * 50)

In [ ]:
# @title CELL 12 (FIXED) — UPLOAD FOLDER TO HUGGING FACE
import os
from huggingface_hub import HfApi, login

# 1. Login (nếu cần)
if "HF_TOKEN_WIRTE" in os.environ:
    login(token=os.environ["HF_TOKEN_WIRTE"])

# 2. Cấu hình Repo
REPO_ID = "An-CNT/doctorai-tamly-lora"
LOCAL_MODEL_PATH = "./lora_model"  # Thư mục đã lưu ở Cell 10

print(f"🚀 Đang đẩy toàn bộ file trong '{LOCAL_MODEL_PATH}' lên repo '{REPO_ID}'...")

# 3. Upload trực tiếp thư mục
api = HfApi()

# Tạo repo nếu chưa có
api.create_repo(repo_id=REPO_ID, exist_ok=True)

# Upload folder
api.upload_folder(
    folder_path=LOCAL_MODEL_PATH,
    repo_id=REPO_ID,
    repo_type="model"
)

print("✅ Đã upload thành công! Bạn có thể kiểm tra tại:")
print(f"https://huggingface.co/{REPO_ID}")